In [26]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import minimize
from collections import Counter
import pickle

# === กำหนดค่าคงที่ ===
n_qubits = 16
param_size = 2 * n_qubits
shots = 2048
max_iter = 100

# === ฟังก์ชันแปลงข้อมูล ===
def float_to_bit(f):
    val = int(f * 15)
    return format(val, "04b")

def features_to_bitstring(x):
    return ''.join([float_to_bit(f) for f in x])

def get_target_distribution(bits_list):
    counter = Counter(bits_list)
    total = sum(counter.values())
    probs = np.zeros(2**n_qubits)
    for b, c in counter.items():
        idx = int(b, 2)
        probs[idx] = c / total
    return probs

# === สร้างวงจร VQC ===
def create_vqc(params):
    qc = QuantumCircuit(n_qubits)
    for i in range(n_qubits):
        qc.ry(params[i], i)
    for i in range(n_qubits - 1):
        qc.cx(i, i + 1)
    for i in range(n_qubits):
        qc.rz(params[i + n_qubits], i)
    qc.measure_all()
    return qc

# === ฟังก์ชันเทรน VQC สำหรับแต่ละ class ===
def train_vqc_for_class(class_id):
    print(f"\n🎯 Training VQC for class {class_id}...")
    
    # Load data
    X, y = load_iris(return_X_y=True)
    X_class = X[y == class_id]
    
    # Normalize
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X_class)

    # Convert to bitstring
    bitstring_target = [features_to_bitstring(x) for x in X_scaled]
    target_probs = get_target_distribution(bitstring_target)

    # Define loss function
    def get_probs(params):
        qc = create_vqc(params)
        backend = Aer.get_backend("qasm_simulator")
        tqc = transpile(qc, backend)
        job = backend.run(tqc, shots=shots)
        counts = job.result().get_counts()
        probs = np.zeros(2**n_qubits)
        for bit, count in counts.items():
            idx = int(bit[::-1], 2)
            probs[idx] = count / shots
        return probs

    def loss(params):
        gen_probs = get_probs(params)
        return np.mean((gen_probs - target_probs)**2)

    # Optimize
    init_params = np.random.uniform(0, 2 * np.pi, param_size)
    result = minimize(loss, init_params, method="COBYLA", options={"maxiter": max_iter})
    trained_params = result.x

    print(f"✅ Finished training class {class_id}")
    return trained_params, scaler


In [27]:
trained_vqcs = {}

# เทรนทุก class (0 = Setosa, 1 = Versicolor, 2 = Virginica)
for class_id in range(3):
    params, scaler = train_vqc_for_class(class_id)
    trained_vqcs[class_id] = {
        "params": params,
        "scaler": scaler
    }

# (optional) บันทึกไฟล์ไว้ใช้งานต่อ
with open("trained_vqc_all_classes.pkl", "wb") as f:
    pickle.dump(trained_vqcs, f)



🎯 Training VQC for class 0...
✅ Finished training class 0

🎯 Training VQC for class 1...
✅ Finished training class 1

🎯 Training VQC for class 2...
✅ Finished training class 2


In [30]:
def generate_data_from_vqc(class_id, trained_vqcs, n_samples=300, max_batch=2048):
    """
    สุ่มข้อมูลจาก VQC ที่เทรนไว้แล้วสำหรับ class นั้น ๆ
    - class_id: int (0, 1, 2)
    - trained_vqcs: dict จาก train_vqc_for_class
    - n_samples: จำนวนข้อมูลที่ต้องการ
    - max_batch: จำนวนครั้งวัดสูงสุดต่อรอบ (จำกัดตาม simulator)
    """
    from qiskit import transpile
    from qiskit_aer import Aer

    params = trained_vqcs[class_id]["params"]
    scaler = trained_vqcs[class_id]["scaler"]

    def bits_to_float(bits):
        return int(bits, 2) / 15.0

    def bitstring_to_features(bs):
        bs = bs[::-1]
        return [bits_to_float(bs[i:i+4]) for i in range(0, 16, 4)]

    def create_vqc(params):
        qc = QuantumCircuit(16)
        for i in range(16):
            qc.ry(params[i], i)
        for i in range(15):
            qc.cx(i, i + 1)
        for i in range(16):
            qc.rz(params[i + 16], i)
        qc.measure_all()
        return qc

    generated = []
    backend = Aer.get_backend("qasm_simulator")

    while len(generated) < n_samples:
        qc = create_vqc(params)
        tqc = transpile(qc, backend)
        job = backend.run(tqc, shots=min(n_samples - len(generated), max_batch))
        counts = job.result().get_counts()

        for bs, count in counts.items():
            f = bitstring_to_features(bs)
            generated.extend([f] * count)

    generated = np.array(generated[:n_samples])
    X_gen = scaler.inverse_transform(generated)
    y_gen = np.full((n_samples,), 7 if class_id == 0 else 8 if class_id == 1 else 9 if class_id == 2 else class_id)
    return X_gen, y_gen


In [31]:
import pandas as pd

# โหลด parameter ที่เทรนไว้ก่อนหน้านี้
with open("trained_vqc_all_classes.pkl", "rb") as f:
    trained_vqcs = pickle.load(f)

# สร้างข้อมูล 300 ตัวจากแต่ละ class
X_list, y_list = [], []
for cid in range(3):
    Xc, yc = generate_data_from_vqc(cid, trained_vqcs, n_samples=300)
    X_list.append(Xc)
    y_list.append(yc)

# รวมทุก class
X_all = np.vstack(X_list)
y_all = np.concatenate(y_list)

# สร้าง DataFrame
df = pd.DataFrame(X_all, columns=["sepal length", "sepal width", "petal length", "petal width"])
df["label"] = y_all
print(df.sample(10))

# บันทึกข้อมูลที่สร้างขึ้น
df.to_csv("6_2_generated_iris_16qubit.csv", index=False)


     sepal length  sepal width  petal length  petal width  label
400          5.74     2.186667          4.82     1.746667      8
842          6.70     3.693333          6.26     2.206667      9
822          7.70     2.306667          6.10     1.473333      9
643          7.90     3.693333          6.10     2.060000      9
14           4.50     2.580000          1.54     0.466667      7
412          6.30     3.213333          3.84     1.213333      8
781          6.50     2.946667          5.30     2.500000      9
388          5.88     3.120000          3.28     1.426667      8
246          4.90     3.840000          1.36     0.533333      7
751          6.70     3.053333          6.10     1.400000      9
